In [17]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three


In [0]:
train = pd.read_pickle('data/train.p')
X_train, y_train = train['features'], train['labels']
test = pd.read_pickle('data/test_road_sign.p')
X_test, y_test = test['features'], test['labels']

if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
df = pd.read_csv('data/signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
def train_model(model, X_train, y_train, params_fit=None):
  if params_fit is None:
    params_fit = {}

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )

  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


def get_cnn_v5(input_shape, num_classes):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(0.3),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),
          
          Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(0.3),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(1024, activation='relu'),
          Dropout(0.3),

          Dense(num_classes, activation='softmax'),
])


In [24]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 15s 56ms/step - loss: 2.5696 - accuracy: 0.2993 - val_loss: 0.7846 - val_accuracy: 0.7566
Epoch 2/5
272/272 [==============================] - 15s 53ms/step - loss: 0.6721 - accuracy: 0.7899 - val_loss: 0.1720 - val_accuracy: 0.9461
Epoch 3/5
272/272 [==============================] - 14s 53ms/step - loss: 0.3053 - accuracy: 0.9090 - val_loss: 0.0751 - val_accuracy: 0.9791
Epoch 4/5
272/272 [==============================] - 15s 54ms/step - loss: 0.1913 - accuracy: 0.9439 - val_loss: 0.0333 - val_accuracy: 0.9907
Epoch 5/5
272/272 [==============================] - 14s 53ms/step - loss: 0.1562 - accuracy: 0.9534 - val_loss: 0.0316 - val_accuracy: 0.9932


0.9655328798185941

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1] 
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [42]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropuout_cnn_block_one': hp.uniform('dropuout_cnn_block_one', 0, 1),
    'dropuout_cnn_block_two': hp.uniform('dropuout_cnn_block_two', 0, 1),
    'dropuout_cnn_block_three': hp.uniform('dropuout_cnn_block_three', 0, 1),
    'dropuout_dense_block_one': hp.uniform('dropuout_dense_block_one', 0, 1),
    'dropuout_dense_block_two': hp.uniform('dropuout_dense_block_two', 0, 1)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropuout_cnn_block_one': 0.6258846304699723, 'dropuout_cnn_block_three': 0.2940400714847331, 'dropuout_cnn_block_two': 0.28500726805570054, 'dropuout_dense_block_one': 0.781652449128994, 'dropuout_dense_block_two': 0.25607771060687856}
accuracy=0.8891156315803528
{'batch_size': 60.0, 'dropuout_cnn_block_one': 0.5981007979817882, 'dropuout_cnn_block_three': 0.9222643697534963, 'dropuout_cnn_block_two': 0.04627723047129806, 'dropuout_dense_block_one': 0.8799230833301767, 'dropuout_dense_block_two': 0.6874236051105613}
accuracy=0.24149659276008606
{'batch_size': 140.0, 'dropuout_cnn_block_one': 0.04711213629251132, 'dropuout_cnn_block_three': 0.07189625830189095, 'dropuout_cnn_block_two': 0.8505102517807805, 'dropuout_dense_block_one': 0.5724864564379227, 'dropuout_dense_block_two': 0.09246693599104594}
accuracy=0.9117913842201233
{'batch_size': 180.0, 'dropuout_cnn_block_one': 0.21302572277035448, 'dropuout_cnn_block_three': 0.03840572844007384, 'dropuout_cnn_block

In [0]:
def get_model(params):
  return Sequential([
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
          MaxPool2D(),
          Dropout(params['dropuout_cnn_block_one']),

          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(params['dropuout_cnn_block_two']),
          
          Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
          Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
          MaxPool2D(),
          Dropout(params['dropuout_cnn_block_three']),

          Flatten(),

          Dense(1024, activation='relu'),
          Dropout(params['dropuout_dense_block_one']),

          Dense(1024, activation='relu'),
          Dropout(params['dropuout_dense_block_two']),

          Dense(num_classes, activation='softmax'),
])